# Advanced API's - Expand Our Project (Wk4 Day4 Homework)

In [ ]:
import requests
import json
import base64
from getpass import getpass

url = 'https://cae-bootstore.herokuapp.com'

endpoint_login = "/login"
endpoint_user = "/user"
endpoint_book = "/book"

class ApiFuncs():
    def get_books():
        books = requests.get(url+endpoint_book)
        return books.json()['books']

    def get_single_book(book_id):
        single_book = requests.get(url+endpoint_book +'/'+str(book_id))
        return single_book.json()

    def register_user(payload):
        payload_json_string = json.dumps(payload)
        headers = {
            'Content-Type':'application/json'
        }
        response = requests.post(
            url + endpoint_user,
            data = payload_json_string,
            headers = headers
        )
        return response.text

    def login_user(user_name, password):
        auth_string = user_name + ":" + password
        
        headers={
            'Authorization' : "Basic "+base64.b64encode(auth_string.encode()).decode()
        }
        user_data = requests.get(
            url + endpoint_login,
            headers=headers
        )
        return user_data.json()

    def edit_user(token, payload):
        payload_json_string = json.dumps(payload)
        headers={
            'Content-Type':'application/json',
            'Authorization':'Bearer ' + token
        }
        response = requests.put(
            url + endpoint_user,
            data=payload_json_string,
            headers=headers
        )
        return response.text

    def delete_user(token):
        headers = {
            'Authorization':"Bearer " + token
        }
        response = requests.delete(
            url+endpoint_user,
            headers=headers
        )
        return response.text


: 

In [ ]:
import ApiFuncs
import time
from IPython.display import clear_output
from IPython.display import Image
from IPython.display import display


class BookFuncs():      
    def get_book_by_id(books, id):
        return list(filter(lambda book: book['id']==id, books))[0]

    def get_category_list(books):
        return {book['subject'].title() for book in books}

    def get_book_by_category(books, category):
        return list(filter(lambda book: book['subject'].title()==category.title(),books))

    def display_book_cover(book):
        cover = Image(url=book['img'])
        display(cover)

    def display_book_short(book):
        print(f"{book['id']} \t| {book['title'][:50].ljust(50)} | \t{book['subject']}")

    def display_book_long(book):
        cover = ApiFuncs.Image(url=book['img'])
        ApiFuncs.display(cover)
        print(f'''
Title:\t {book['title']}
Author: {book['author']}
Pages:\t {book['pages']}
Subject: {book['subject']}
Summary: {book['summary']}
''')

    def show_small_book_list(self):
        if not self.reading_list:
            print("Your reading list is empty")
        else:
            for book in self.reading_list:
                BookFuncs.display_book_short(book)

    def browse_books(books, reading_list, subject=None):
        while True:
            clear_output()
            print(f'''
Welcome to the Book Browser
You are viewing {subject if subject else 'all'} books
[ID] \t| {"[TITLE]".ljust(50)} | [SUBJECT]
        ''')
            if subject:
                books= BookFuncs.get_book_by_category(books, subject)
            for book in books:
                BookFuncs.display_book_short(book)

            selection = input("Select your book by ID or Type 'BACK' to back out.")
            if selection.lower()=='back':
                return
            elif selection.isnumeric() and int(selection) in map(lambda book: book['id'], books):
                selection=int(selection)
                selected_book = BookFuncs.get_book_by_id(books, selection)
                while True:
                    print(f'''
You Selected: {selected_book['title']}
1. Add Book To Reading List
2. View More Information
3. Go Back
4. Go To Main Menu
''')
                    action = input("Action: ")
                    if action =="1":
                        reading_list.add_book(selected_book)
                        print("As you wish")
                        time.sleep(3)
                        break
                    elif action =="2":
                        clear_output()
                        BookFuncs.display_book_long(selected_book)
                        input("Press Enter To Continue")
                    elif action=="3":
                        break
                    elif action=="4":
                        return
                        
            else:
                print("Invalid ID")
                time.sleep(2)
                continue

class ReadingList():
    def __init__(self):
        self.reading_list=[]
        
    def add_book(self, book):
        if book not in self.reading_list:
            self.reading_list.append(book)
    
    def remove_book(self, book):
        self.reading_list.remove(book)
    
    def empty(self):
        self.reading_list=[]
    
    def show_book_list(self):
        clear_output()
        if not self.reading_list:
            print("Your reading list is empty")
        for book in self.reading_list:
            print(f'''
{"="*50}            
\n
Title:\t {book['title']}
Book ID: {book['id']}
Author:\t {book['author']}
Subject: {book['subject']}
Summary: {book['summary']}
\n
{"="*50}            
\n
''') 

: 

## Display Images in Jupyter notebook

In [ ]:
from IPython.display import Image, display
image = Image(url='https://s2982.pcdn.co/wp-content/uploads/2018/11/commuter-pig-keeper-book-cover.jpg')
display(image)
print('hello')

: 

: 

: 

## Make an Application

In [ ]:
import ApiFuncs
import BookFuncs
from getpass import getpass
import time
from IPython.display import clear_output


class UI():
    def login(email):
        clear_output()
        password = getpass("Password: ")
        user = ApiFuncs.login_user(email, password) 
        print(user)
        time.sleep(3)
        return user

    def delete_account(user):
        ApiFuncs.delete_user(user['token'])
        time.sleep(4)

    def register():
        clear_output()
        print("Registration:")
        email = input("Email: ")
        first_name = input("First Name: ")
        last_name = input("Last Name: ")
        password = input("Password: ")
        
        user_dict={
            "email":email,
            "first_name":first_name,
            "last_name":last_name,
            "password":password
        }
        return ApiFuncs.register_user(user_dict)

    def edit(user):
        prompt = input('Type in One of The Following Edit Options: First Name, Last Name, or Password to Edit.\n')
        if prompt.lower() == "first name":
            first = input('Enter a New First Name: \n')
            edit_payload={
            "first_name":first,           
            }
            return ApiFuncs.edit_user(user['token'], edit_payload)            
            
        elif prompt.lower() == 'last name':
            last = input('Enter a New Last Name: \n')
            edit_payload={
            "last_name":last,           
            }
            return ApiFuncs.edit_user(user['token'], edit_payload)

        elif prompt.lower() == 'password':
            password = input('Enter a New Password: \n')
            edit_payload={
            "password":password,           
            }            
            return ApiFuncs.edit_user(user['token'], edit_payload)

def main():
    reading_list = BookFuncs.ReadingList()
#    books = ApiFuncs.get_books()
    user = {}
    
    while True:
        clear_output()
        print("Welcome to the Bookstore")
        email = input("Type Your Email to Login or Type `register` to Register ")
        if email == 'register':
            success_register = UI.register()
            if success_register:
                print(f"{user['first_name'].title()}, you have successfully registered")
                time.sleep(3)
                continue
            else:
                print("Failed to register")
                time.sleep(3)
        elif email.lower() == "quit":
            print(f"Goodbye {user['first_name'].title()}")
            break
        else:
            try:
                user = UI.login(email)
            except:
                print("Invalid Username/Password combo")
                time.sleep(2)
                continue
        # First Scene of our app (above)

        while True:
            clear_output()
            print("""
Welcome to the Repository            
You can:            
1. Browse All Books
2. Browse Book by Category
3. View Reading List
4. Remove Book From Reading List
5. Edit Account    
6. Delete Account
7. Quit
""")
            command = input(f"Hi, {user['first_name'].title()} - Please Choose an Option. ")
            if command == "1":
                BookFuncs.browse_books(books, reading_list)

            elif command == "2":
                while True:
                    print(" | ".join(BookFuncs.get_category_list(books)))
                    cat = input("Category: ").title()
                    if cat in BookFuncs.get_category_list(books):
                        BookFuncs.browse_books(books, reading_list, cat)
                        break
                    print("Invalid Category")
                    time.sleep(2)
                    continue

            elif command == "3":
                reading_list.show_book_list()
                input("Press Enter To Return")
                continue

            elif command == "4":
                while True:
                    clear_output()
                    reading_list.show_book_list()
                    garbage = input(f"What Book ID Would You Like to Remove, {user['first_name'].title()}? or Enter 'BACK' to back out ")
                    if garbage.lower() == "back":
                        break
                    elif garbage.isnumeric() and int(garbage) in map(lambda book: book['id'], reading_list.reading_list):
                        reading_list.remove_book(list(filter(lambda book: book['id']==int(garbage), reading_list.reading_list))[0])
                        print(f'{garbage} has been removed')
                        time.sleep(2)
                        break
                    else:
                        print(f"{user['first_name'].title()}, {garbage} is Not in Your Collection")
                        time.sleep(2)
                        break
                continue   
                    
            elif command == "5":
                if UI.edit(user):
                    print('Your Edit Was Successful.')
                    time.sleep(2)
                    continue

            elif command == "6":
                prompt = input("Type 'delete' to Delete Your Account.\n")
                if prompt.lower() == "delete":
                    prompt = input('Please Enter Email.\n')
                    if UI.delete_account(user):
                        print(f"Sorry to See You go {user['first_name'].title()}, Bye-Bye!")
                    break
                else:
                    continue

            elif command == "7":
                print("Goodbye")
                break
            else:
                print("Invalid Selection")
                time.sleep(2)
                continue
        break   


main() 

: 

: 